In [15]:
# 加载 .env 到环境变量
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [6]:
import os
_documents_folder = os.getenv("LANGCHAIN_CHINESE_DOCUMENTS_FOLDER")

In [31]:
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_openai import OpenAI, OpenAIEmbeddings

In [ ]:
!poetry add lancedb

In [32]:
def get_file_extension(filename: str) -> str:
    return filename.split(".")[-1]

In [33]:
class FileLoadFactory:
    @staticmethod
    def get_loader(filename: str):
        filename = filename.strip()
        ext = get_file_extension(filename)
        if ext == "pdf":
            return PyPDFLoader(filename)
        elif ext == "docx":
            return Docx2txtLoader(filename)
        else:
            raise NotImplementedError(f"File extension {ext} not supported.")

In [34]:
import os
_documents_folder = os.getenv("LANGCHAIN_CHINESE_DOCUMENTS_FOLDER")
# current_working_directory = os.getcwd()
# print("当前工作目录的绝对路径:", current_working_directory)

def get_files(directory, extensions):
  files = []
  for dirpath, dirnames, filenames in os.walk(directory):
    for filename in filenames:
      if os.path.splitext(filename)[1] in extensions:
        files.append(os.path.join(dirpath, filename))
  return files

# 使用示例
get_files(_documents_folder, ['.docx', '.pdf'])

['/Users/xuehongwei/github/wencheng/data/documents/gd_tax_arch.docx']

In [35]:
def load_docs(filename: str) -> List[Document]:
    file_loader = FileLoadFactory.get_loader(filename)
    pages = file_loader.load_and_split()
    return pages

In [41]:
files = get_files(_documents_folder, ['.docx', '.pdf'])

raw_docs = load_docs(files[0])
text_splitter = RecursiveCharacterTextSplitter(
                        chunk_size=1000,
                        chunk_overlap=200,
                        length_function=len,
                        add_start_index=True,
                    )
documents = text_splitter.split_documents(raw_docs)

In [50]:
print(documents[22].page_content)

在长期的业务应用建设过程中，没有统一的技术规范要求，建设的应用系统技术多样，差异化明显，系统应用升级方式多样，难以进行统一标准化的管理，系统应用升级方法和工具参差不齐，亟待规范统一系统开发、测试、部署等技术。

（四）数据中心级的容灾能力亟待改善

机构改革后，以税务总局的数据灾备要求文件为指导，合并的数据中心未形成整体容灾机制。从容灾能力要素分析，目前缺乏跨数据中心层级的备用数据处理系统、备用网络系统、备用基础设施、专业技术支持能力、运行维护管理能力以及灾难恢复预案；从业务应用需求分析，未对业务系统进行容灾需求评估分析，缺乏业务系统容灾指标（RPO恢复点目标/RTO恢复时间目标）。

当前以南海、五山为主要的生产数据中心，海珠、江门为备份数据中心。南海、五山各自数据中心内主机故障恢复能力不完善，当整个数据中心发生火灾、网络瘫痪等灾难事件时，没有数据中心能够提供灾难恢复能力，无法保障业务连续运转。灾难恢复能力是保障业务连续的基石，亟需建立灾难恢复策略和完善灾难恢复能力。

组织管理

近年来，广东省税务局的信息化建设取得骄人的成绩，但在信息化建设自主可控能力、信息化人才管理、信息化制度、信息化创新能力四个方面仍有持续改进与提升的空间。主要问题总结如下：

（一）信息化建设自主可控能力有待提升

信息化建设大多采用项目制且外部购买服务的建设策略，内部以抽调人员临时组建项目攻关团队的形式进行配合与承接，整体上存在项目管理被动、系统迭代不及时等弊端，在信息化项目的开发、运维、管控上缺少一定的主导权。

（二）信息化人才有待扩充

广东省税务局现有的信息化人才主要分为两类：一是从信息化部门组成来看，人员构成包括征管科技处、税收大数据和风险管理局、信息中心人员和部分由市（区）税务局调派的人员；二是从信息化项目规划、需求、设计、实施和运维来看，规划、需求和设计阶段以广东省税务局内部人员为主，开发实现与运维阶段以第三方供应商为主。


In [ ]:
def ingest_document(filename: str, lanceFile: str):
  """将文档内容向量化，并存放LanceDB"""
  

In [ ]:
def ask_docment(
        filename: str,
        query: str,
) -> str:
    """根据一个PDF文档的内容，回答一个问题"""

    raw_docs = load_docs(filename)
    if len(raw_docs) == 0:
        return "抱歉，文档内容为空"
    text_splitter = RecursiveCharacterTextSplitter(
                        chunk_size=1000,
                        chunk_overlap=200,
                        length_function=len,
                        add_start_index=True,
                    )
    documents = text_splitter.split_documents(raw_docs)
    if documents is None or len(documents) == 0:
        return "无法读取文档内容"
    db = Chroma.from_documents(documents, OpenAIEmbeddings(model="text-embedding-ada-002"))
    qa_chain = RetrievalQA.from_chain_type(
        llm=OpenAI(
            temperature=0,
            model_kwargs={
                "seed": 42
            },
        ),  # 语言模型
        chain_type="stuff",  # prompt的组织方式
        retriever=db.as_retriever()  # 检索器
    )
    response = qa_chain.invoke(query+"(请用中文回答)")
    return response